# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [41]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [42]:
from pyspark.sql.functions import col, count, countDistinct

Create our session as described in the tutorials

In [43]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [44]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [45]:
citations.show(5)

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows



In [46]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [47]:
patents.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   null|     BE|   null|    null|      1|  null|   269|  6|    69| null|       1|    null|    0.0|    null|    null|    null|    null|    null|    null|    null|
|3070802| 1963| 1096|   null|     US|     TX|    null|      1|  null|     2|  6|    63| null|       0|    null|   null|    null|    null|    null|    null|    null|    null|    null|
|3070803| 1963| 1096|   null|     US|     IL|    null|      1|  null|     2|  6|    6

**My solution below: First we need to create a table with patents and their states, as well as clean up null values**

In [54]:
new_pats = patents.select("PATENT", "POSTATE").filter( patents["POSTATE"] != "null")
new_pats.show(5)

+-------+-------+
| PATENT|POSTATE|
+-------+-------+
|3070802|     TX|
|3070803|     IL|
|3070804|     OH|
|3070805|     CA|
|3070806|     PA|
+-------+-------+
only showing top 5 rows



Now we inner join citations and and new patent table on CITED as the key s the RDD implementation

In [56]:
joined = citations.join(new_pats, citations.CITED == new_pats.PATENT) # Inner join by default, per lecture
joined.show(5)

+-------+-------+-------+-------+
| CITING|  CITED| PATENT|POSTATE|
+-------+-------+-------+-------+
|3949375|3071753|3071753|     MN|
|4120573|3071753|3071753|     MN|
|4271479|3071753|3071753|     MN|
|4345315|3071753|3071753|     MN|
|4496943|3071753|3071753|     MN|
+-------+-------+-------+-------+
only showing top 5 rows



Once again we want, (CITING, CITED, CITED STATE)

In [69]:
state_col = col("POSTATE").alias("CITED_STATE") # Pass in the alias for the column since it's actually the Cited State in that column

# states_tbl is the df containing cited state

states_tbl = joined.select("CITING", "CITED", state_col)
states_tbl.show(5)

+-------+-------+-----------+
| CITING|  CITED|CITED_STATE|
+-------+-------+-----------+
|3949375|3071753|         MN|
|4120573|3071753|         MN|
|4271479|3071753|         MN|
|4345315|3071753|         MN|
|4496943|3071753|         MN|
+-------+-------+-----------+
only showing top 5 rows



Now we need the Citing State by joining the above with original patent table again except this time on CITING as key to get CITING STATE

In [70]:
# citing_st is the df containing the citing state

citing_st = states_tbl.join(new_pats, states_tbl.CITING == new_pats.PATENT)

# We need to drop the PATENT column that will arise from this join

st_tbl = citing_st.drop('PATENT')

# We create and pass in the alias for the CITING STATE column
citing_st_col = col('POSTATE').alias('CITING_STATE')
new_tbl = st_tbl.select("CITING", citing_st_col, "CITED", "CITED_STATE")
new_tbl.show(5)

+-------+------------+-------+-----------+
| CITING|CITING_STATE|  CITED|CITED_STATE|
+-------+------------+-------+-----------+
|3858597|          MT|3815160|         NY|
|3858597|          MT|3675252|         AZ|
|3859029|          NY|3741706|         OH|
|3859029|          NY|3685936|         OH|
|3859627|          MI|3368197|         MI|
+-------+------------+-------+-----------+
only showing top 5 rows



Lastly we will need to find and count same state citations, as well as add that count to a df

In [72]:
same_state_tbl = new_tbl.where( new_tbl.CITING_STATE == new_tbl.CITED_STATE)
same_state_tbl.show(5)

+-------+------------+-------+-----------+
| CITING|CITING_STATE|  CITED|CITED_STATE|
+-------+------------+-------+-----------+
|4178878|          AK|3464385|         AK|
|3974004|          AL|3745074|         AL|
|3974004|          AL|3585090|         AL|
|3974004|          AL|3762972|         AL|
|3974004|          AL|3692600|         AL|
+-------+------------+-------+-----------+
only showing top 5 rows



In [94]:
# Now aggregate by grouping on the key and counting

# We're using .agg() to be able to then count same state occurrences
count_same = same_state_tbl.groupby('CITING').agg(count(same_state_tbl.CITING_STATE == same_state_tbl.CITED_STATE).alias('SAME_STATE')).orderBy('SAME_STATE',ascending=False)
count_same.show(10)

+-------+----------+
| CITING|SAME_STATE|
+-------+----------+
|5959466|       125|
|5983822|       103|
|6008204|       100|
|5952345|        98|
|5958954|        96|
|5998655|        96|
|5936426|        94|
|5925042|        90|
|5739256|        90|
|5978329|        90|
+-------+----------+
only showing top 10 rows



Last, we will need to append the count column to the original new_pats table. 

ie: We can can inner join original patents with our "count_same" table where CITING matches PATENT and we order by "SAME_STATE" which is the count

In [95]:
# Joining ORIGINAL patents with same state count on the CITING # as the key, ordering count descending, and showing

result = patents.join(count_same, patents.PATENT == count_same.CITING).orderBy('SAME_STATE',ascending=False)
result.show(10)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+----------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD| CITING|SAME_STATE|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+----------+
|5959466| 1999|14515|   1997|     US|     CA|    5310|      2|  null|   326|  4|    46|  159|       0|     1.0|   null|  0.6186|    null|  4.8868|  0.0455|   0.044|    null|    null|5959466|       125|
|5983822| 1999|14564|   1998|     US|     TX|  569900|      2|  null|   114|  5|    55|  200|       0|   0.995|   null|  0.7201|    null|   12.45|     0.0|     0.0|    null|    null|5983822|  